In [1]:
import pandas as pd
import os
import glob
import geopy.distance
import matplotlib.pyplot as plt
import requests
import math
import numpy as np

In [2]:
def read_data(folder): 
  '''
  Pasamos el path del folder donde están los datos dinámicos
  '''
  merged_data = pd.DataFrame() 
  csv_files = glob.glob(os.path.join(folder, "*.csv"))

  for f in csv_files:  
      df = pd.read_csv(f)
      merged_data = pd.concat([merged_data,df])
  return merged_data

def segment_data(data, path_ice, path_ev):
  '''
  Pasamos los datos dinámicos (ya leídos) y los paths a los csv estáticos
  
  '''
  ice_hev = pd.read_csv(path_ice, sep = ';') 
  phev_ev = pd.read_csv(path_ev, sep = ';')
  hev_id = ice_hev.loc[ice_hev['Vehicle Type']=='HEV']
  ice_id = ice_hev.loc[ice_hev['Vehicle Type']=='ICE']
  ev_id = phev_ev.loc[phev_ev['EngineType']=='EV']
  phev_id = phev_ev.loc[phev_ev['EngineType']=='PHEV']
  ev = pd.merge(data,ev_id,on ='VehId', how = 'inner')
  phev = pd.merge(data,phev_id,on ='VehId', how = 'inner')
  hev = pd.merge(data,hev_id,on ='VehId', how = 'inner')
  ice = pd.merge(data,ice_id,on ='VehId', how = 'inner')
  return ev, phev, hev, ice

data = read_data('/content/drive/MyDrive/Monitoría EV/Data')
path_ice, path_ev ='/content/drive/MyDrive/Monitoría EV/Static Data/VED_Static_Data_ICE&HEV.csv','/content/drive/MyDrive/Monitoría EV/Static Data/VED_Static_Data_PHEV&EV.csv'
ev, phev, hev, ice = segment_data(data,path_ice,path_ev)


Limpiar datos carros eléctricos

In [3]:
ev.drop(columns = ['MAF[g/sec]','Engine RPM[RPM]','Absolute Load[%]','Short Term Fuel Trim Bank 1[%]','Short Term Fuel Trim Bank 2[%]', 
                   'Long Term Fuel Trim Bank 1[%]', 'Long Term Fuel Trim Bank 2[%]','Fuel Rate[L/hr]', 'Air Conditioning Power[kW]', 'Transmission',
                   'EngineType', 'Drive Wheels', 'Engine Configuration & Displacement', 'Vehicle Class'], inplace = True)
ev.fillna(method='ffill')

,DayNum,VehId,Trip,Timestamp(ms),Latitude[deg],Longitude[deg],Vehicle Speed[km/h],OAT[DegC],Air Conditioning Power[Watts],Heater Power[Watts],HV Battery Current[A],HV Battery SOC[%],HV Battery Voltage[V],Generalized_Weight
0,1.719774,10,1558,0,42.277066,-83.763404,53.590000,5.0,0.0,2250.0,-21.5,96.341469,386.0,3500
1,1.719774,10,1558,200,42.277066,-83.763404,51.980000,5.0,0.0,2250.0,-21.5,96.341469,386.0,3500
2,1.719774,10,1558,1200,42.277066,-83.763404,50.369999,5.0,0.0,2250.0,-21.5,96.341469,386.0,3500
3,1.719774,10,1558,1500,42.277066,-83.763404,50.369999,5.0,0.0,2250.0,23.5,96.341469,390.5,3500
4,1.719774,10,1558,2300,42.277066,-83.763404,49.799999,5.0,0.0,2250.0,23.5,96.341469,390.5,3500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476303,208.990283,541,1153,108800,42.285123,-83.801513,66.449997,30.0,0.0,0.0,-13.5,59.024395,376.5,3500
476304,208.990283,541,1153,109600,42.285123,-83.801513,66.400002,30.0,0.0,0.0,-13.5,59.024395,376.5,3500
476305,208.990283,541,1153,109800,42.285123,-83.801513,66.400002,30.0,0.0,0.0,-14.0,59.024395,376.5,3500
476306,208.990283,541,1153,110700,42.285123,-83.801513,66.320000,30.0,0.0,0.0,-14.0,59.024395,376.5,3500


In [4]:
#Para ahorrar RAM
del phev
del hev
del ice

Creamos índice separador

In [5]:
def create_idx(data):
  i = 0
  j = 0
  indx = []
  idx = 0
  prevtrip = data['Trip'][0]
  pendiente = False
  for trip in zip(data['Trip']):
    if pendiente:
      idx += 1
      pendiente = False
    if prevtrip!=trip:
      pendiente = True
    indx.append(idx)
    prevtrip = trip
  return indx

indices = create_idx(ev)
ev.insert(len(ev.columns),'Indx',indices)

Creamos variable de potencia instántanea

In [6]:
def suma_riemann(data):
  potencia = []
  previdx = indices[0]
  prevtime = data['Timestamp(ms)'][0]
  pot = 0
  for time,curr,volt,idx in zip(data['Timestamp(ms)'][1:],data['HV Battery Current[A]'][1:],data['HV Battery Voltage[V]'][1:],data['Indx'][1:]):
    if previdx != idx:
      potencia.append(pot)
      pot = 0
    else:
      pot += (time-prevtime)*curr*volt
    previdx = idx
    prevtime = time
  potencia.append(pot) 
  return pd.DataFrame(potencia, columns = ['Potencia instantanea'])

potencia = suma_riemann(ev)

Creamos variable de aceleración

In [7]:
def acceleration(data):
  acel = []
  previdx = indices[0]
  prevtime = data['Timestamp(ms)'][0]
  prevspeed = data['Vehicle Speed[km/h]'][0]
  temp = []
  for time,speed,idx in zip(data['Timestamp(ms)'][1:],data['Vehicle Speed[km/h]'][1:],data['Indx'][1:]):
    if previdx != idx:
      if temp:
        acel.append(np.mean(temp))
      else:
        acel.append(0)
      temp = []
    else:
      if time != prevtime:
        temp.append((speed-prevspeed)/(time-prevtime))
    previdx = idx
    prevspeed = speed
    prevtime = time
  acel.append(np.mean(temp)) 
  return pd.DataFrame(acel, columns = ['Mean accel'])

acel = acceleration(ev)

Creamos otras variables con la función groupby de Python

In [8]:
means = ev.groupby(ev['Indx']).mean().loc[:,['OAT[DegC]','Vehicle Speed[km/h]']]
vars = ev.groupby(ev['Indx']).var().loc[:,['Vehicle Speed[km/h]']]
firsts = ev.groupby(ev['Indx']).first().loc[:,['Latitude[deg]','Longitude[deg]']]
lasts = ev.groupby(ev['Indx']).last().loc[:,['Latitude[deg]','Longitude[deg]']]

vars.columns = ['Variance speed']
means.columns = ['Mean OAT', 'Mean speed']
firsts.columns = ['First lat', 'First lon']
lasts.columns = ['Last lat', 'Last lon']
new_df = pd.concat([means,vars,potencia,acel,firsts,lasts], axis=1)

Creamos variable de distancia

In [9]:
def dists(data):
  dists = []
  for prevlat,prevlon,lat,lon in zip(data['First lat'],data['First lon'],data['Last lat'], data['Last lon']):
    dists.append(geopy.distance.geodesic((prevlat,prevlon),(lat,lon)).m)
  return pd.DataFrame(dists, columns = ['Distance'])
distancia = dists(new_df)
new_df = pd.concat([new_df,distancia],axis=1)

Calculamos la elevación

In [12]:
from re import T
def funcion(df1, API_endpoint, inicio, fin):
    data=[]
    for i in range(inicio,fin):
        data.append([df1['First lat'][i],df1['First lon'][i]])
    x = 0
    while x != 200:
        r = requests.post(url = API_endpoint, json= data)
        x = r.status_code
    str = r.content.decode("utf-8")
    str = str.replace('[','')
    str = str.replace(']','')
    lista = list(np.float_(str.split(',')))
    for i in range (inicio,fin):
        df1['elevation_2'][i]=lista[i-inicio]
    return(df1)

API_ENDPOINT = "https://elevation.racemap.com/api"
new_df.insert(len(new_df.columns),'elevation',[np.nan]*len(new_df))
new_df = funcion(new_df,API_ENDPOINT,0,len(new_df))

Creamos variable de pendiente

In [15]:
def slope(data):
  slope = []
  prevelev = data['elevation'][0]
  for elev in data['elevation'][1:]:
    slope.append(elev-prevelev)
    prevelev = elev
  slope.append(0)
  return pd.DataFrame(slope, columns = ['Slope'])
pendiente = slope(new_df)
new_df = pd.concat([new_df,pendiente],axis = 1)
new_df.drop(columns='elevation',inplace=True)

Limpiar datos

In [20]:
new_df = new_df.dropna(axis = 1)
new_df.drop(columns = ['First lat'], axis =1, inplace=True)
new_df.drop(columns = ['First lon'], axis =1, inplace=True)
new_df.drop(columns = ['Last lat'], axis =1, inplace=True)
new_df.drop(columns = ['Last lon'], axis =1, inplace=True)
new_df.to_csv('content\out.csv')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
#SE CAE POR RAM
from scipy.spatial.distance import mahalanobis
import scipy as sp
def cut_outliers(data):
  Sx = data.cov().values
  invcov = sp.linalg.inv(Sx)
  mean = data.mean().values
  x_minus_mu = data-mean
  left_term = np.dot(x_minus_mu,invcov)
  mahal_nueva = np.dot(left_term,x_minus_mu.T)
  dist = np.diag(mahal_nueva)
  p = np.percentile(dist,90)
  data.insert(len(data.columns), 'Mahal', dist)
  data = data[data['Mahal']<p]
  data.drop(columns = ['Mahal'], axis=1, inplace=True)
  return data
cut_outliers(new_df)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Mean OAT,Mean speed,Variance speed,Potencia instantanea,Mean accel,Distance,Slope
0,4.996154,35.504307,269.558553,4.705748e+08,0.000143,2754.833171,-25.142439
1,4.762172,22.884194,252.638086,-1.578312e+09,0.000117,1505.030065,13.046000
2,9.324251,19.366348,191.046801,1.736762e+09,-0.000102,1133.082475,13.868992
3,9.621479,26.449436,302.456960,-2.985813e+09,0.000284,643.797603,-26.914992
4,14.154018,24.018816,257.000846,3.944926e+09,-0.000445,2185.121789,13.639000
...,...,...,...,...,...,...,...
499,6.742608,61.625577,481.545170,-2.383904e+09,-0.000032,4330.817148,-12.256488
500,7.020356,51.684426,581.744774,1.673396e+09,0.000269,5621.606829,-2.286512
501,31.436170,34.083069,676.307542,2.989142e+09,-0.000013,703.827894,11.000000
502,30.738351,47.413153,578.934733,1.804928e+08,0.000573,1336.447709,-1.294000


Entrenamiento de modelos

In [24]:
from sklearn.linear_model import LinearRegression, QuantileRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model, svm
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
y = new_df['Potencia instantanea']
X = new_df.drop(columns='Potencia instantanea')
X_train, X_test, y_train, y_test = train_test_split(X, y)
reg = LinearRegression().fit(X_train, y_train)
print('Linear Regression Score: ' , reg.score(X_train, y_train))
predicted = reg.predict(X_test)
print('Linear Regression MSE: ' , mean_squared_error(y_test, predicted))
regressor = DecisionTreeRegressor(random_state=0)
print('Score: ', cross_val_score(regressor, X, y, cv=10))
#reg = svm.SVR()
#reg.fit(X_train, y_train)
#print('SVM Regression Score: ', reg.score(X_train,y_train))
#predicted = reg.predict(X_test)
#print('SVM Regression MSE: ' , mean_squared_error(y_test, predicted))


Linear Regression Score:  0.13343202815642685
Linear Regression MSE:  7.629272140000543e+19
Score:  [-1.38064706 -1.42642416 -2.67542491 -2.58735569 -0.14472252 -2.02456152
 -1.33838236 -0.70106692 -1.00738617 -0.76767874]


In [25]:
corrm = new_df.corr()
corrm

,Mean OAT,Mean speed,Variance speed,Potencia instantanea,Mean accel,Distance,Slope,Mahal
Mean OAT,1.000000,0.124319,0.038848,-0.043845,0.053093,0.029799,-0.025371,0.062746
Mean speed,0.124319,1.000000,0.383876,-0.158308,-0.134640,0.278841,-0.167413,0.160481
Variance speed,0.038848,0.383876,1.000000,0.044909,-0.044915,0.219030,-0.045458,0.279280
Potencia instantanea,-0.043845,-0.158308,0.044909,1.000000,0.032704,0.131215,0.032795,0.218844
Mean accel,0.053093,-0.134640,-0.044915,0.032704,1.000000,-0.049904,0.005856,-0.020194
Distance,0.029799,0.278841,0.219030,0.131215,-0.049904,1.000000,-0.026525,0.365550
Slope,-0.025371,-0.167413,-0.045458,0.032795,0.005856,-0.026525,1.000000,0.032983
Mahal,0.062746,0.160481,0.279280,0.218844,-0.020194,0.365550,0.032983,1.000000
